# Enhanced Information Retrieval with Langchain, OpenAI, DuckDuckGo, Wikipedia, and PythonREPL

In [1]:
import os
os.environ['OPENAI_API_KEY'] = 'your-api-key' #paste your api key here  

In [2]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI


In [6]:
llm = ChatOpenAI(model_name = 'gpt-4-turbo', temperature = 0)

template = '''
Answer the following question as best you can.
Question: {q}
'''

prompt_template = PromptTemplate.from_template(template)
prompt = hub.pull('hwchase17/react')


In [7]:
# python Repl tool (for executing python code)
python_repl = PythonREPLTool()
python_repl_tool = Tool(
    name= 'Python REPL',
    func = python_repl.run,
    description= 'Use when you need to use python to answer the question. You should input python code'
)

# Wikipedia tool (for searching wikipedia)
api_wrapper = WikipediaAPIWrapper()
wikipedia = WikipediaQueryRun(api_wrapper= api_wrapper)
wikipedia_tool = Tool(
    name='Wikipedia',
    func=wikipedia.run,
    description= 'Use when you need to lookup for topic, country or person on wikipedia'
)

# DuckDuckGo search tool (for general web search tool)
search = DuckDuckGoSearchRun()
duckduckgo_tool = Tool(
    name='DuckDuckGo search',
    func= search.run,
    description= 'Useful from when you need to perform internet search to find information that another tools cannot provide'
)


tools = [python_repl_tool, wikipedia_tool, duckduckgo_tool]

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose= True,
    handle_parsing_errors= True,
    max_iterations= 10
)


In [8]:
question = input('write your question:')
output = agent_executor.invoke({
    'input': prompt_template.format(q=question)
})

write your question: generate the first 20 numbers in Fibonacci series




> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


The Fibonacci series is a sequence where each number is the sum of the two preceding ones, usually starting with 0 and 1. I can generate this sequence using a simple Python program.

Action: Python REPL

Action Input:
```python
def fibonacci(n):
    fib_series = [0, 1]
    while len(fib_series) < n:
        fib_series.append(fib_series[-1] + fib_series[-2])
    return fib_series[:n]

fibonacci(20)
```
[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181]

Thought: The Python code successfully generated the first 20 numbers in the Fibonacci series.

Final Answer: The first 20 numbers in the Fibonacci series are: 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181.

> Finished chain.


In [9]:
question = input('write your question:')
output = agent_executor.invoke({
    'input': prompt_template.format(q=question)
})

write your question: who is the current prime minister of India?




> Entering new AgentExecutor chain...
To find the current Prime Minister of India, I can use Wikipedia to get the most accurate and updated information.

Action: Wikipedia

Action Input: "Prime Minister of India"
Page: Prime Minister of India
Summary: The prime minister of India (ISO: Bhārata kē/kī pradhānamaṁtrī) is the head of government of the Republic of India. Executive authority is vested in the prime minister and his chosen Council of Ministers, despite the president of India being the nominal head of the executive.  The prime minister has to be a member of one of the houses of bicameral Parliament of India, alongside heading the respective house. The prime minister and his cabinet are at all times responsible to the Lok Sabha.
The prime minister is appointed by the president of India; however, the prime minister has to enjoy the confidence of the majority of Lok Sabha members, who are directly elected every five years, lest the prime minister shall resign. The prime minister 

In [10]:
output['output']

'The current Prime Minister of India is Narendra Modi, serving since 26 May 2014.'

In [11]:
question = input('write your question:')
output = agent_executor.invoke({
    'input': prompt_template.format(q=question)
})

write your question: tell me about Napoleon Bonaparte early life in short




> Entering new AgentExecutor chain...
To provide information about Napoleon Bonaparte's early life, I can use the Wikipedia tool to access a concise summary of his early years.

Action: Wikipedia

Action Input: "Napoleon Bonaparte"
Page: Jérôme Napoléon Bonaparte
Summary: Jérôme Napoléon "Bo" Bonaparte (5 July 1805 – 17 June 1870) was an American farmer, chairman of the Maryland Agricultural Society, first president of the Maryland Club, and the son of Elizabeth Patterson and Jérôme Bonaparte, brother of Napoleon I.

Page: Prince Napoléon-Jérôme Bonaparte
Summary: Prince Napoléon Joseph Charles Paul Bonaparte (9 September 1822 – 17 March 1891), usually called Napoléon-Jérôme Bonaparte or Jérôme Bonaparte, was the second son of Jérôme, King of Westphalia, youngest brother of Napoleon I, and his second wife Catharina of Württemberg. Following the death of his nephew Louis-Napoléon, Prince Imperial in 1879, he claimed headship of the House of Bonaparte until his death in 1891. An outspo

In [12]:
output['output']

'Napoleon Bonaparte was born on August 15, 1769, on the island of Corsica to a family of Italian origin. He moved to mainland France in 1779 and was commissioned as an officer in the French Army in 1785. Napoleon supported the French Revolution in 1789 and promoted its cause in Corsica. His rapid rise in military ranks began after he broke the siege of Toulon in 1793 and suppressed royalist insurgents in Paris in 1795. This set the stage for his prominent role in French and European history.'